# **GR5242 Final Exam: Text Prediction with LSTM networks**

**Instructions**: This problem is an individual assignment -- you are to complete this problem on your own, without conferring with your classmates.  You should submit a completed and published notebook to Courseworks; no other files will be accepted.

## Description:

This homework exercise has 3 primary goals:
 * Introduce some basic concepts from natural language processing
 * Get some practice training recurrent neural networks, specifically on text data
 * Be able to generate fake text data from your favorite author!   

By the end of this exercise, you will have a basic, but decent, computer program which can simulate the writing patterns of any author of your choice.

Here is an outline of the rest of the exercise.
 1. Data loading
     - We will start by downloading a text from Project Gutenberg that we will try to model
     - Data preprocessing and numerical encoding
     - Making training `Dataset` and `DataLoader` objects
 3. Learn to generate text with a neural network
     - Defining the recurrent network
     - Training
     - Predicting and sampling text from the model

     There are 12 questions (70 points) in total, which include coding and written questions. You can only modify the codes and text within \### YOUR CODE HERE ### and/or \### YOUR ANSWER HERE ###.


In [1]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import re

## Character-level language modeling

Our goal here is to build a model of language letter-by-letter. Since we may also allow numbers, spaces, and punctuation, it's better to say character-by-character. We will start by fixing an "alphabet": the set of allowed characters.

In math notation, let's call the alphabet $A$. In code,

In [6]:
alphabet = " abcdefghijklmnopqrstuvwxyz1234567890.,!?:;ABCDEFGHIJKLMNOPQRSTUVWXYZ\n"

# Section 1: Data loading and preprocessing

We will start by loading our data. Data will be provided with the assignment.

If you are having issues with those data or would like another dataset to work with, you can also try downloading training data from Project Gutenberg: https://www.gutenberg.org/. Project Gutenberg is a free repository of public domain books. Find any book you like, and download it in Plain Text UTF-8 format.

For example, we can use Shakespeare's complete works: https://www.gutenberg.org/ebooks/100. There is a link on that page to the Plain Text format data. 

Download the .txt file, and then upload it from your computer to colab (click at left on the File icon, then click the upload icon).  

*Important*: whichever work you choose, make sure you have enough data! The size of your plain text file should be at least 2MB.

In [15]:
# after uploading your file to Colab, set this variable to its path:
txt_path = "C:/Users/zha/Downloads/test_text.txt"

Let's load the text and see what it says:

In [24]:
with open(txt_path, encoding="utf-8") as txt_file:
    text = txt_file.read()

print("text is", len(text), "characters long.")
print()
print("A sample from the middle:")
print()
print(text[len(text) // 2 : len(text) // 2 + 100])

text is 2648423 characters long.

A sample from the middle:

there was another reason why the theory of verbal inflation didn’ t seem
to hold water: it was only 


### Data standardization

Now, we will clean the data: removing extra spaces and linebreaks, and getting rid of characters which are not in our alphabet.

In [25]:
# remove extra characters by replacing them with spaces
text = re.sub(rf"[^{alphabet}]", " ", text)

Let's see how it looks again:

In [26]:
print(text[len(text) // 2 : len(text) // 2 + 100])
x_prompt = text[len(text) // 2 : len(text) // 2 + 1000]

there was another reason why the theory of verbal inflation didn  t seem
to hold water: it was only 


### Numerical encoding

Unfortunately, neural networks don't understand text. So, we need to convert our characters to numerical values. Here are some helper functions for doing this.

In [27]:
# let's build a dictionary mapping characters to integers
char2int = {c: i for i, c in enumerate(alphabet)}
alphabet_array = np.array([c for c in alphabet])

# this function will turn a string into a numpy array of integers
def int_encode(string):
    if any(c not in char2int for c in string):
        raise ValueError(
            "Found a character which was not in the alphabet in the input "
            f"to int_encode. Valid alphabet characters: {alphabet}"
          )
    return np.array([char2int[c] for c in string])

# this function will decode a numpy array of integers back to a string
def int_decode(int_array):
    return ''.join(alphabet_array[int_array])

(Question 1a: 4 points) Test out `int_encode` by passing `test_string` in and printing the result.

In [36]:
# Let's test these out!
### YOUR CODE HERE ###
test_int = int_encode(x_prompt)
test_int

array([20,  8,  5, 18,  5,  0, 23,  1, 19,  0,  1, 14, 15, 20,  8,  5, 18,
        0, 18,  5,  1, 19, 15, 14,  0, 23,  8, 25,  0, 20,  8,  5,  0, 20,
        8,  5, 15, 18, 25,  0, 15,  6,  0, 22,  5, 18,  2,  1, 12,  0,  9,
       14,  6, 12,  1, 20,  9, 15, 14,  0,  4,  9,  4, 14,  0,  0, 20,  0,
       19,  5,  5, 13, 69, 20, 15,  0,  8, 15, 12,  4,  0, 23,  1, 20,  5,
       18, 41,  0,  9, 20,  0, 23,  1, 19,  0, 15, 14, 12, 25,  0, 14,  5,
        3,  5, 19, 19,  1, 18, 25,  0, 20, 15,  0, 19,  1, 12, 20,  0, 20,
        8,  5,  0,  9, 14,  4,  9, 22,  9,  4, 21,  1, 12,  0, 21, 14,  9,
       20, 19,  0, 15,  6,  0, 20,  8,  5,  0, 23, 15, 18,  4, 69,  7, 18,
       15, 21, 16,  0,  0, 18,  5,  4,  5, 13, 16, 20,  9, 15, 14,  0,  0,
       23,  9, 20,  8,  0,  1,  0, 19, 13,  1, 12, 12, 38,  0,  9, 14, 14,
       15,  3,  5, 14, 20,  0, 12,  1,  3, 11,  0, 15,  6,  0,  7, 18,  1,
       22,  9, 20, 25,  0, 38,  0,  1, 14,  4,  0, 20,  8,  5, 25, 69,  9,
       14, 19, 20,  1, 14

(Question 1b: 4 points) Decode the result from the last cell using `int_decode` to make sure it is the same as `test_string`

In [35]:
### YOUR CODE HERE ###
int_decode(test_int)== x_prompt

True

Is the decoding the same as `test_string`? It should -- you have a bug above if not.

### Make a training dataset

First, we make a numerical encoded version of the entire dataset:

In [37]:
enctext = int_encode(text)

Use `torch.tensor` to make it into a PyTorch tensor:

In [38]:
enctext = torch.tensor(enctext)
print(enctext)

tensor([60, 57, 44,  ..., 46, 63, 69], dtype=torch.int32)


# Section 2: Training a NN

Our model will work as follows:
 - One-hot encoded input gets passed into a linear embedding layer. These two operations are combined with the `Embedding` layer: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html
 - LSTM cell
 - Linear decoder layer

PyTorch has two main ways of interfacing with recurrent networks. In the case of LSTMs, those are:
 - the LSTM layer https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
 - the LSTMCell layer https://pytorch.org/docs/stable/generated/torch.nn.LSTMCell.html

Both models are sequential: the goal is to process a batch of sequences of input features and produce a batch of sequences of output features. The `LSTM` class makes this simple and easy, and the `LSTMCell` class gives more control by allowing you to process the sequences one element at a time. We will use the `LSTM` layer to keep things simple, but keep in mind that some of what we do could be made more efficient with `LSTMCell`.

The inputs and outputs to recurrent networks in `pytorch` have shape: `(batch_dimension, sequence_dimension, feature_dimension)`. In this case, our feature dimension is `len(alphabet)`.

Something to keep in mind: the output of this network will be stateful! In each batch, the `k`th output along the sequence dimension will be the logits for predicting the `k+1`th input in the batch.

In [39]:
# We will use this constant below
HIDDEN_DIM = 128

In [40]:
# Defining some parameters about data batching, explained in the next section
# Note: after you get the entire assignment working, you can make these
# bigger and train for longer, to get better performance
SEQUENCE_LENGTH = 32
BATCH_SIZE = 16

### Making the dataset of (input, target) pairs

To train the model, we need to make a [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) containing input and target sequences. Our input sequences will be sequences of length `SEQUENCE_LENGTH` containing int-encoded characters from the input. Our target sequences will be the "next characters" corresponding to the input sequence: so, if the input sequence is the 10th, 11th, ... characters, then the target sequence is the 11th, 12th, ... characters.

We will walk through creating these.

(Question 2a: 8 points) Write a `batch` function for a `torch` tensor, which we defined above, to make disjoint consecutive sequences of consecutive characters of length `SEQUENCE_LENGTH`. These disjoint sequences should become the rows of a two-dimensional tensor.

[`torch.split()`](https://pytorch.org/docs/stable/generated/torch.split.html) and [`torch.vstack()`](https://pytorch.org/docs/stable/generated/torch.vstack.html) may be useful. You may also write this function recursively.

Remember to be careful of the edge case that arises when `len(enctext) % SEQUENCE_LENGTH != 0`.

In [76]:
def batch(enctext, SEQUENCE_LENGTH):
    # YOUR CODE HERE
    if len(enctext) % SEQUENCE_LENGTH != 0:
        return torch.vstack(torch.split(enctext, SEQUENCE_LENGTH)[:-1])
    else:
        return torch.vstack(torch.split(enctext, SEQUENCE_LENGTH))
    pass

(Question 2b: 8 points) Now, use batch to create input and target sequences which have been offset by 1 element:

In [77]:
input_seqs  = batch(enctext, SEQUENCE_LENGTH) # YOUR CODE HERE
target_seqs = batch(enctext[1:], SEQUENCE_LENGTH) # YOUR CODE HERE

(Question 2c: 6 points) Now, use the `torch` builtin class [`torch.utils.data.TensorDataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset) to create a dataset called `pairs` of (input, target) pairs:

In [79]:
pairs = TensorDataset(input_seqs, target_seqs) # YOUR CODE HERE
pairs

(Question 2d: 4 points) Finally, define a `torch.utils.data.DataLoader` object to generate batches of pairs of length `BATCH_SIZE`:

In [83]:
train_loader = DataLoader(pairs, BATCH_SIZE, shuffle=True) # YOUR CODE HERE
train_loader

You may uncomment the below cell if you would like to understand the structure of the `train_loader`.

In [90]:
for i, (x, y) in enumerate(train_loader):
    print(x, y)
    if i > 1:
        break

tensor([[ 5, 19, 38,  0, 20,  8,  1, 20,  0,  3, 12,  1,  9, 13,  0, 13, 21, 19,
         20,  0,  5, 14, 10, 15, 25,  0, 20,  8,  5,  0, 16, 18],
        [ 6,  0, 20,  8,  1, 20,  0, 20,  1, 11,  5, 19,  0, 15, 14,  0, 20,  8,
          5,  0, 19,  8,  1, 16,  5,  0, 15,  6, 69,  5, 22,  5],
        [ 8,  5,  0,  9, 14,  1,  2,  9, 12,  9, 20, 25,  0, 15,  6,  0,  8, 21,
         13,  1, 14,  0,  2,  5,  9, 14,  7, 19,  0, 20, 15,  0],
        [19, 16, 15, 14, 19,  9,  2,  9, 12,  9, 20, 25,  0, 37,  0, 43, 14,  4,
          0, 19, 15,  0, 20,  8,  5, 25,  0,  6,  9,  5, 18,  3],
        [ 5, 25,  5,  0,  2,  1, 18,  5, 12, 25,  0, 16,  5, 18,  3,  5,  9, 22,
          5,  4,  0,  2,  5,  3,  1, 21, 19,  5,  0,  9, 20,  0],
        [13, 19, 20,  1, 14,  3,  5, 19,  0,  1, 14, 25, 15, 14,  5,  0,  9, 19,
         69,  3,  1, 16,  1,  2, 12,  5,  0, 15,  6,  0,  1, 14],
        [12, 25,  0, 18,  5, 13,  1,  9, 14, 19,  0,  9, 19,  0,  6, 15, 18, 13,
         21, 12,  1, 19, 37, 69, 65, 

(Question 2e: 10 points) Model definition: the basic structure should be a [`nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) model with an [`nn.Embedding`](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) layer with input dimension `len(alphabet)` and output dimension `HIDDEN_DIM`, followed by an [`nn.LSTM`](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) layer with `HIDDEN_DIM` features, followed by a [`nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) layer with `len(alphabet)` features. Make sure that your `nn.LSTM` layer is initialized with `batch_first=True`.

There are two complications here. The first is that the `nn.LSTM` layer returns a tuple of `(output, (recurrent state))`. In our model output, we only need the `output` tensor. The second is that the `nn.LSTM` layer returns the entire sequence along its second dimension. To deal with this, define a layer that handles both of these issues within the `pytorch` framework.

More specifically, write a class called `extract_tensor` as a subclass of [`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). This class should contain the required `__init__()` method, and a `forward()` method that extract the `output` tensor from the tuple returned by the `nn.LSTM` layer and returns this `output`. We will additionally have the `extract_tensor` layer store an attribute named `return_sequences`. If `return_sequences == True`, it should return the entire output. Otherwise, it should return `output[:,-1,:]`.

We will toggle the `return_sequence` attribute to be `True` for training and `False` for prediction. This necessitates the use of an `OrderedDict` in defining our model so that our layers have names to reference (this can be a useful technique in general if you would like to manipulate individual layers of a model).

In general, this type of problem is something you can often handle by making simple, self-defined layers like this.

This `extract_tensor` layer should be placed between the `nn.LSTM` and `nn.Linear` layers in the `nn.Sequential` model.

In [162]:
from collections import OrderedDict

return_sequences = True

class extract_tensor(nn.Module):
    def __init__(self, return_sequences):
        super(extract_tensor, self).__init__()
        self.model = nn.Identity() # YOUR CODE HERE
    
    def forward(self, x):
#         tensor, _ = x
        tensor, _ = self.model(x)# YOUR CODE HERE
        if return_sequences:
            return tensor
        else:
            return tensor[:,-1,:]
        
layer_ordered_dict = OrderedDict([
    ('embedding', nn.Embedding(len(alphabet),HIDDEN_DIM)),
    ('LSTM', nn.LSTM(HIDDEN_DIM, HIDDEN_DIM, 1, batch_first=True)),
    ('extract', extract_tensor(return_sequences)),
    ('linear', nn.Linear(HIDDEN_DIM, len(alphabet)))])

model = nn.Sequential(layer_ordered_dict) # YOUR CODE HERE

(Question 2f: 2 points) If we want to use the output of the model as logits for predicting a character (which we can think of as a class), what loss should we use? Name this `criterion`. Additionally, define an optimizer to use in training. As per usual, we will recommend the use of `optim.Adam`.

In [163]:
criterion = nn.CrossEntropyLoss() # YOUR CODE HERE
optimizer = optim.Adam(model.parameters()) # YOUR CODE HERE

(Question 2g: 8 points) Train the model!
Note:

1. We will be restricting the number of iterations rather than number of epochs. A recommended, reasonable value to start with is NUM_ITER = 8000. Use a while loop outside of the usual for loop used to iterate through the DataLoader, and make sure to break the for loop if you have exceeded the prescribed number of iterations.
2.   Given the above parameters, this training should take about 2 minutes.  
3.   Performance will improve but will only be mediocre.  For this exercise, that is adequate.  
4. However, to really see what this model can do, you can (should!) set a larger number of batches above and a larger hidden state so that you can take many epochs of the data.  If you train for ~10hrs on a model with 256 hidden states (as we show in class), performance can be quite good.  This step is not required for the assignment, but please do try it on your own.



In [164]:
### YOUR CODE HERE ###
NUM_ITER = 8000

i = 0
while i < NUM_ITER:
    ####### REMOVE ########
#     break                 # this prevents an error if running all cells
    ####################### before having filled in this part of the code
    losses = []
    for batch in train_loader:
        if i >= NUM_ITER:
            break
        i += 1
        
        x_batch, y_batch = batch
        # YOUR CODE HERE - TO DO: zero the gradient and extract your x_batch, y_batch
        optimizer.zero_grad()

        y_logit = model(x_batch)
        
        y_logit = y_logit.permute(0, 2, 1) # transposing the non-batch dimensions for correct calculation

        y_batch = F.one_hot(y_batch.long()) # YOUR CODE HERE: translate into one-hot encoding using F.one_hot
        
        y_batch = y_batch.float().permute(0, 2, 1)
        
        loss = criterion(y_logit, y_batch) # YOUR CODE HERE: calculate the loss

        # YOUR CODE HERE: backpropagate and use the optimizer to take a step
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        
        if i >= 100:
            print('iteration', i, '- loss:', loss.item(),
                  '- avg of last 100:', sum(losses[-100:])/100, end='\r')
        else:     
            print('iteration', i, '- loss:', loss.item(), end='\r')

Here, make sure the loss goes down as it trains.

# Section 3: Did it work? Let's see what the model learned

Here, we'll write some functions to see how well the model has learned to predict text and to draw samples from the model.

First, we'll give you a function to "seed" the model with some input text and then predict the most likely future text. It will be your job to create a variation on this function in the question below, so make sure you understand how it works.

In [203]:
def predict(seed_string, sample_length=50):
  # Convert seed_string to int
    current_text_ints = list(int_encode(seed_string))
    

    for i in range(sample_length):
        # Add an empty batch dimension and convert to tensor
        text_arr = np.array(current_text_ints).reshape(1, -1)
        text_arr = torch.tensor(text_arr)

        # set our model to return only one output instead of the sequence
        model.extract.return_sequences = False

        # Get the full sequence of predictions, remove the batch dim
        logits = model(text_arr[-1])

        # Remove the batch dimension and get the final logits
        final_logits = logits[-1]

        # Get the prediction using tf.argmax
        pred = torch.argmax(final_logits)

        # Append this to `current_text_ints`
        current_text_ints.append(pred.numpy())

    return int_decode(np.array(current_text_ints))

In [389]:
test_seed = "The novel centers on "
print(predict(test_seed, 50))

The novel centers on the probably the probably the probably the probabl


In [32]:
test_seed = "The novel centers on "
print(predict(test_seed, 50))

The novel centers on the same the same the same the same the same the s


In [196]:
# feel free to try your own seed!

It seems like maybe the model learned something, but the output is a little boring. Let's make it more interesting with *randomness*!

Right now, the function always picks the most likely next letter. Instead, let's sample the next letter from the model's predicted probability distribution.

(Question 3a: 8 points) Fill in the blanks in the function below.

In [385]:
def generate(seed_string, sample_length=50):
  # Convert seed_string to int
    current_text_ints = list(int_encode(seed_string))

    for i in range(sample_length):
    # Add an empty batch dimension and convert to tensor
        text_arr = np.array(current_text_ints).reshape(1, -1)
        text_arr = torch.tensor(text_arr)

        # set our model to return only one output instead of the sequence
        model.extract.return_sequences = False

        # Get the full sequence of predictions, remove the batch dim
        logits = model(text_arr[-1])

        # Remove the batch dimension and get the final logits
        final_logits = logits[-1]

        # Normalize the final_logits to a probability distribution
        probs = F.softmax(final_logits, dim = -1) # YOUR CODE HERE

        # Call .numpy so we can use a numpy function
        probs = probs.detach().numpy()

        # Sample from the probability distribution using
        # the function np.random.choice
        sample = np.random.choice(a = len(alphabet), p = probs)  # YOUR CODE HERE

        # Append this to `current_text_ints`
        current_text_ints.append(sample)

    return int_decode(np.array(current_text_ints))

(Question 3b: 6 points) Test this function `generate`. Is its output different from `predict` when given the same seed? How does it differ, and why?

In [392]:
# YOUR CODE HERE
print(generate(test_seed))
print()
print(generate(text[len(text) // 2 : len(text) // 2 + 100]))
print()
print(text[len(text) // 2 : len(text) // 2 + 500])

The novel centers on ranged the mirrospless
we all he flagmainst, which

there was another reason why the theory of verbal inflation didn  t seem
to hold water: it was only and ally really too, in the earty years much leate

there was another reason why the theory of verbal inflation didn  t seem
to hold water: it was only necessary to salt the individual units of the word
group  redemption  with a small, innocent lack of gravity , and they
instantly came trippingly from the tongue.  Y ou ve just saved my soul!  or
some such; who has not said something of the sort at one time or another ,
provided of course that it refers to nothing more than the relief after a ten 
minute wait or some equally slight inconvenience t


In [398]:
print('''Yes, the output is different from predict as we can observe that predict has repeated chunk of words i.e., "the probably",
while generate does not. The reason is that, in predict case, we only select the most likely next letter, so every letter's next letter
is fixed and there are only finite number of characters in our alphabet.''')

Yes, the output is different from predict as we can observe that predict has repeated chunk of words i.e., "the probably",
while generate does not. The reason is that, in predict case, we only select the most likely next letter, so every letter's next letter
is fixed and there are only finite number of characters in our alphabet.


(Question 3c: 2 point) Try running `generate` a few times with the same seed. Are the results the same or different? Why?

In [39]:
print('''So let us not place any particular value on the city’ s name. Like all big
cities it was made up of irregularity , change, forward spurts, failures to keep
step, collisions of objects and interests, punctuated by unfathomable
silences; made up of pathways and untrodden ways, of one great rhythmic
beat as well as the chronic discord and mutual displacement of all its
contending rhythms.''')

So let us not place any particular value on the city’ s name. Like all big
cities it was made up of irregularity , change, forward spurts, failures to keep
step, collisions of objects and interests, punctuated by unfathomable
silences; made up of pathways and untrodden ways, of one great rhythmic
beat as well as the chronic discord and mutual displacement of all its
contending rhythms.


In [399]:
print('''The results are different since we have a randomness in selecting next letter, so even with the same probability,
we might choose different letter each time''')

The results are different since we have a randomness in selecting next letter, so even with the same probability,
we might choose different letter each time
